In [8]:
import time
import hmac
import hashlib
import json
import requests
import random

def upload_ei(_name_label, _values, hmac_key, api_key):
    HMAC_KEY = hmac_key
    API_KEY = api_key
    emptySignature = ''.join(['0'] * 64)

    # Assuming Fs is your sampling frequency in Hz
    Fs = 1000
    Ts = (1 / Fs) * 1000

    data = {
        "protected": {
            "ver": "v1",
            "alg": "HS256",
            "iat": time.time()  # epoch time, seconds since 1970
        },
        "signature": emptySignature,
        "payload": {
            "device_name": "ac:87:a3:0a:2d:1b",
            "device_type": "NANO33BLE",
            "interval_ms": Ts,
            "sensors": [
                { "name": "Volts", "units": "adu/mv" }
            ],
            "values": _values
        }
    }

    # Encode in JSON
    encoded = json.dumps(data)

    # Sign message
    signature = hmac.new(bytes(HMAC_KEY, 'utf-8'), msg=encoded.encode('utf-8'), digestmod=hashlib.sha256).hexdigest()

    # Set the signature again in the message, and encode again
    data['signature'] = signature
    encoded = json.dumps(data)

    # Upload the file
    res = requests.post(
        url='https://ingestion.edgeimpulse.com/api/training/data',
        data=encoded,
        headers={
            'Content-Type': 'application/json',
            'x-file-name': _name_label,
            'x-api-key': API_KEY
        }
    )

    if res.status_code == 200:
        print('Uploaded file to Edge Impulse', res.status_code, res.content)
    else:
        print('Failed to upload file to Edge Impulse', res.status_code, res.content)

# Reemplaza estos valores con los tuyos
HMAC_KEY = "2b5313c3c1094cfc38f66d1b8fe4a489"
API_KEY = "ei_407bf3377a0382faf3b6542dfc38027ce1371a3a7a8a0d44"
NAME_LABEL = "EMG1"
FILE_PATH = r"C:\ISB\data\emg_cesar.txt"


# Función para verificar si una línea puede convertirse a un float
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Leer los valores del archivo TXT, ignorando líneas no numéricas
with open(FILE_PATH, 'r') as file:
    values = [list(map(float, line.strip().split())) for line in file if is_float(line.strip().split()[0])]

# Dividir los datos en fragmentos de 1000 muestras cada uno
chunk_size = 1000
chunks = [values[i:i + chunk_size] for i in range(0, len(values), chunk_size)]

# Subir cada fragmento como un archivo separado
for i, chunk in enumerate(chunks):
    if len(chunk) == chunk_size:  # Asegurarse de que solo suba fragmentos completos de 1000 muestras
        file_name = f"EMG_1{i+1}.json"
        upload_ei(file_name, chunk, HMAC_KEY, API_KEY)

Uploaded file to Edge Impulse 200 b'EMG_11.json.50rmj71u.json'
Uploaded file to Edge Impulse 200 b'EMG_12.json.50rmj8mh.json'
Uploaded file to Edge Impulse 200 b'EMG_13.json.50rmjad0.json'
Uploaded file to Edge Impulse 200 b'EMG_14.json.50rmjc5b.json'
Uploaded file to Edge Impulse 200 b'EMG_15.json.50rmjdu2.json'
Uploaded file to Edge Impulse 200 b'EMG_16.json.50rmjfpd.json'
Uploaded file to Edge Impulse 200 b'EMG_17.json.50rmjhdj.json'
Uploaded file to Edge Impulse 200 b'EMG_18.json.50rmjivv.json'
Uploaded file to Edge Impulse 200 b'EMG_19.json.50rmjkkm.json'
Uploaded file to Edge Impulse 200 b'EMG_110.json.50rmjm80.json'
Uploaded file to Edge Impulse 200 b'EMG_111.json.50rmjnsf.json'
Uploaded file to Edge Impulse 200 b'EMG_112.json.50rmjpd6.json'
Uploaded file to Edge Impulse 200 b'EMG_113.json.50rmjr1d.json'
Uploaded file to Edge Impulse 200 b'EMG_114.json.50rmjsil.json'
Uploaded file to Edge Impulse 200 b'EMG_115.json.50rmju45.json'
Uploaded file to Edge Impulse 200 b'EMG_116.json.

KeyboardInterrupt: 